## Learning Notebook:  Build a simple RAG workflow using Open-Source tools

### Overview

This tutorial demonstrates how to implement **Retrieval-Augmented Generation (RAG)** using an **open-source Large Language Model (LLM)** and **Pinecone** as a vector database.

### Context

This notebook highlights the most important steps needed to build a simple RAG pipeline using open source tools form Langchain, sentence_transformers, and pincone. 

### Goal:

- **process documents** (TXT, PDF, Word) for RAG
- **embed and store** text in Pinecone
- **retrieve relevant information** from the database
- **generate responses** using an **open-source LLM** (Llama-2, GPT4All, etc.)


Follow the Jupyter notebook below and answer the tasks marked as ✅ Task for Students.


#### **🔧 Step 0: collect some documents**
✅ Task for Students: collect and add some documents in ./doc in the same directory of this notebook. For example, you can download modul descriptions of different study programs in Frankfurt UAS or the slides-decks of Data Mining course.

#### **🔧 Step 1: Install Required Libraries**
✅ Task for Students: try to install the required dependencies on your laptop/PC

#### **📂 Step 2: Load and Prepare Documents (TXT, PDF, Word)**
read documents from a folder (`docs/`) and prepare them for the next steps

In [3]:
import os
import fitz  # PyMuPDF for PDF processing
import docx

# Function to extract text from different document formats
def extract_text_from_file(file_path):
    if file_path.endswith('.txt'):
        with open(file_path, 'r', encoding='utf-8') as file:
            return file.read()
    elif file_path.endswith('.pdf'):
        text = ''
        with fitz.open(file_path) as pdf:
            for page in pdf:
                text += page.get_text()
        return text
    elif file_path.endswith('.docx'):
        doc = docx.Document(file_path)
        return '\n'.join([para.text for para in doc.paragraphs])
    else:
        return None  # Unsupported format

In [4]:
# Load documents from the 'docs/' folder
def load_documents(folder_path='docs/'):
    documents = []
    for filename in os.listdir(folder_path):
        file_path = os.path.join(folder_path, filename)
        text = extract_text_from_file(file_path)
        if text:
            documents.append(text)
    return documents

docs = load_documents()

#### **📂 Step 3: chunk the loaded Documents with the RecursiveCharacterTextSplitter from Langchain**
use a chunk size of 500 and chunk overlap of 50

In [9]:
# Split documents into chunks
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)

chunks = text_splitter.split_text(' '.join(docs))

print(f'Loaded {len(chunks)} text chunks.')

Loaded 13663 text chunks.


#### **🔎 Step 4: Embed chunks and Store in Pinecone**
We use `sentence-transformers` to convert text into **vector embeddings** and store them in Pinecone.

In [10]:
api_key_dir = "/Users/LarryDinh/Documents/code_keys/Pinecone_API_key.txt"
with open(api_key_dir, "r", encoding="utf-8") as file:
    API_KEY = file.read()

In [11]:
from pinecone import Pinecone, ServerlessSpec

# Initialize Pinecone
pc = Pinecone(api_key=API_KEY)

In [12]:
index_name = "rag-tutorial"
dimension = 384  # Dimension of the model we're using (e.g., all-MiniLM-L6-v2)

# Create index if it doesn't exist
if not pc.has_index(index_name):
    pc.create_index(
        name=index_name,
        dimension=dimension,
        metric="cosine",
        spec=ServerlessSpec(cloud="aws", region="us-east-1")
    )

# Connect to the index
index = pc.Index(index_name)

In [14]:
import torch
print("PyTorch:", torch.__version__)



PyTorch: 2.9.1


In [13]:
import sentence_transformers
print("sentence-transformers version:", sentence_transformers.__version__)


sentence-transformers version: 5.1.2


In [14]:
import logging
from sentence_transformers import LoggingHandler

logging.basicConfig(
    format="%(asctime)s - %(message)s",
    level=logging.INFO,
    handlers=[LoggingHandler()]
)

In [15]:
from sentence_transformers import SentenceTransformer

# Load embedding model: Loads or creates a SentenceTransformer model that can be used to map sentences / text to embeddings.
embedder = SentenceTransformer("all-MiniLM-L6-v2")  # 384-dim output

2025-11-20 20:55:47,275 - Use pytorch device_name: mps
2025-11-20 20:55:47,287 - Load pretrained SentenceTransformer: all-MiniLM-L6-v2


In [16]:
# Create vector records for the chunks
vectors = [
    {
        "id": str(i),
        "values": embedder.encode(chunk).tolist(),
        "metadata": {"text": chunk}
    }
    for i, chunk in enumerate(chunks)
]

Batches: 100%|██████████| 1/1 [00:00<00:00, 93.72it/s]


In [17]:
# Upsert = Update values and metadata if exist, Insert if not. This allows safe re-run of code without creating duplicates.
# Upsert in small batches to avoid payload limit
batch_size = 50
for i in range(0, len(vectors), batch_size):
    batch = vectors[i:i + batch_size]
    index.upsert(batch)

print(f"✅   Inserted {len(vectors)} vectors into '{index_name}'.")

✅   Inserted 13663 vectors into 'rag-tutorial'.


#### **🤖 Step 5: Load an Open-Source LLM for Answer Generation**
OllamaLLM is a LangChain-compatible interface for working with Ollama — a tool that allows you to run large language models (like LLaMA) locally.

In [18]:
from langchain_ollama import OllamaLLM
# The follwoing code creates an instance of the LLM interface, pointing to a local Ollama model called "llama3.2".  
model = OllamaLLM(model="llama3.2")

#### **🤖 Step 6: write and embedd your question**

In [19]:
question = "Who is the professor of the course Climate Change & Environmental Impacts ?"

# Embed question
question_vector = embedder.encode(question).tolist()

Batches: 100%|██████████| 1/1 [00:00<00:00,  2.08it/s]


#### **🔄 Step 7: Retrieve Relevant Information & Generate Response**
We retrieve **top-k** relevant documents and use them for answering questions.

In [20]:
# Query Pinecone
results = index.query(
    vector=question_vector,
    top_k=10,
    include_metadata=True
)

In [21]:
#  Prepare retrieved context
contexts = [match["metadata"]["text"] for match in results["matches"]]
context_text = "\n\n".join(contexts)

# Build prompt
prompt = f"""Answer the question based only on the context below.

Context:
{context_text}

Question: {question}
Answer:"""

In [22]:
prompt

'Answer the question based only on the context below.\n\nContext:\nUpon completion of the module, students will \nhave gathered a basic understanding of the weather, climate and climate change \nunderstand the interaction between climate and land use/cover \nhave learned about the impacts of climate and climate change on biodiversity \nhave an enhanced comprehension of climate change mitigation and adaption and a more de-\ntailed knowledge of sustainable management instruments \n3 \nContent\n\n- to analyse the future climate projections; \n- to recognize and evaluate the possible practical mitigation and climate change adaptation \noptions, to identify and to link the affected fields of action, sectors and actors; \n- to analyse the interactions and feedbacks between measures, to analyse the current scientific \nstate-of-the-art on environmental consequences of possible climate change mitigation and \nadaptation measures;\n\nOral and written presentations or written exam \n7 \nConditio

In [26]:
from langchain_ollama import OllamaLLM

model = OllamaLLM(model="llama3.2")
print(model.invoke("Say hello in one short sentence."))

2025-11-20 21:17:21,474 - HTTP Request: POST http://127.0.0.1:11434/api/generate "HTTP/1.1 200 OK"
Hello!


In [27]:
# Query local LLM via Ollama to get an answer
response = model.invoke(prompt)

2025-11-20 21:17:27,668 - HTTP Request: POST http://127.0.0.1:11434/api/generate "HTTP/1.1 200 OK"


In [28]:
response

'Prof. Dr. Oleg Panferov and Prof. Dr. Elke Hietel are the professors for the course, but no one is specifically mentioned as the professor of the course "Climate Change & Environmental Impacts". However, it can be inferred that Prof. Dr. Michael Rademacher is the module beauftragte (module coordinator) for this course.'

In [31]:
#QUESTION 2
question2 = 'What are Proper heating and ventilation in the file "Information FOlder for Residents"?'

# Embed question
question2_vector = embedder.encode(question2).tolist()
# Query Pinecone
results2 = index.query(
    vector=question2_vector,
    top_k=10,
    include_metadata=True
)
#  Prepare retrieved context
contexts2 = [match["metadata"]["text"] for match in results2["matches"]]
context2_text = "\n\n".join(contexts2)

# Build prompt
promp2 = f"""Answer the question based only on the context below.

Context:
{context2_text}

Question: {question2}
Answer:"""

print(promp2)

# Query local LLM via Ollama to get an answer
response2 = model.invoke(promp2)
print(response2)

Batches: 100%|██████████| 1/1 [00:00<00:00, 23.63it/s]


Answer the question based only on the context below.

Context:
Wenn Sie übers Wochenende oder während der Semester-
ferien Ihr Zimmer/Apartment nicht nutzen, stellen Sie die 
Heizung auf Frostschutzstellung (-Symbol). 
Für die Umwelt und Ihr Konto
For the environment and your bank account
Proper heating and airing 
saves a lot of money
 
The importance of proper heating and airing is widely 
underestimated. Proper ventilation brings in new fresh 
air, diminishes humidity and prevents mold formation. 
Thus, ventilation is important, especially after showe-

Stunden aufzuheizen. Zum Temperaturempfinden trägt 
auch die Wärmestrahlung der Wände bei.
Wenn Sie übers Wochenende oder während der Semes-
terferien Ihr Zimmer/Apartment nicht nutzen, stellen Sie 
die Heizung auf Frostschutzstellung . 
Nicht nur heiße Luft …
	
	
	
	
Not just hot air …
Proper heating and ventilation
The importance of proper heating and  
ventilation is widely underestimated.  
Proper ventilation brings in new fres